<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Lab-4.2.2:-Feature-Selection" data-toc-modified-id="Lab-4.2.2:-Feature-Selection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Lab 4.2.2: Feature Selection</a></span><ul class="toc-item"><li><span><a href="#1.-Forward-Feature-Selection" data-toc-modified-id="1.-Forward-Feature-Selection-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. Forward Feature Selection</a></span><ul class="toc-item"><li><span><a href="#1.1-Load-Wine-Data-&amp;-Define-Predictor-and-Target" data-toc-modified-id="1.1-Load-Wine-Data-&amp;-Define-Predictor-and-Target-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>1.1 Load Wine Data &amp; Define Predictor and Target</a></span></li><li><span><a href="#1.2-Overview-of-the-code-below" data-toc-modified-id="1.2-Overview-of-the-code-below-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>1.2 Overview of the code below</a></span></li><li><span><a href="#Code-variables" data-toc-modified-id="Code-variables-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Code variables</a></span></li><li><span><a href="#Methods-of-the-LinearRegression-object-to-investigate" data-toc-modified-id="Methods-of-the-LinearRegression-object-to-investigate-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Methods of the <code>LinearRegression</code> object to investigate</a></span></li><li><span><a href="#Adjusted-$R^2$-formula" data-toc-modified-id="Adjusted-$R^2$-formula-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>Adjusted $R^2$ formula</a></span></li><li><span><a href="#Linear-Regression-reference" data-toc-modified-id="Linear-Regression-reference-1.1.6"><span class="toc-item-num">1.1.6&nbsp;&nbsp;</span>Linear Regression <a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression" target="_blank">reference</a></a></span></li></ul></li></ul></li></ul></div>

![alt text](https://i.imgur.com/1WaY7aA.png)

## Lab 4.2.2: Feature Selection

In [1]:
# import data wrangling and visualisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

%matplotlib inline

### 1. Forward Feature Selection

> Forward Selection: Forward selection is an iterative method in which we start with having no feature in the model. In each iteration, we keep adding the feature which best improves our model till an addition of a new variable does not improve the performance of the model.

Create a Regression model using Forward Feature Selection by looping over all the features adding one at a time until there are no improvements on the prediction metric ( R2  and  AdjustedR2  in this case).

#### 1.1 Load Wine Data & Define Predictor and Target

In [2]:
## Load the wine quality dataset

# Load the wine dataset from csv
wine = pd.read_csv('../../data/winequality_merged.csv')

# define the target variable (dependent variable) as y
y = wine['quality']

# Take all columns except target as predictor columns
predictor_columns = [c for c in wine.columns if c != 'quality']
# Load the dataset as a pandas data frame
X = pd.DataFrame(wine, columns = predictor_columns)

In [3]:
## Create training and testing subsets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#### 1.2 Overview of the code below

The external `while` loop goes forever until there are no improvements to the model, which is controlled by the flag `changed` (until is **not** changed).
The inner `for` loop goes over each of the features not yet included in the model and calculates the correlation coefficient. If any model improves on the previous best model then the records are updated.

#### Code variables
- `included`: list of the features (predictors) that were included in the model; starts empty.
- `excluded`: list of features that have **not** been included in the model; starts as the full list of features.
- `best`: dictionary to keep record of the best model found at any stage; starts 'empty'.
- `model`: object of class LinearRegression, with default values for all parameters.

#### Methods of the `LinearRegression` object to investigate
- `fit()`
- `fit.score()`

#### Adjusted $R^2$ formula
$$Adjusted \; R^2 = 1 - { (1 - R^2) (n - 1)  \over n - k - 1 }$$

#### Linear Regression [reference](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

In [4]:
## Flag intermediate output

show_steps = True   # for testing/debugging
# show_steps = False  # without showing steps

In [5]:
## Use Forward Feature Selection to pick a good model

# start with no predictors
included = []
# keep track of model and parameters
best = {'feature': '', 'r2': 0, 'a_r2': 0}
# create a model object to hold the modelling parameters
model = LinearRegression() # create a model for Linear Regression
# get the number of cases in the test data
n = X_test.shape[0]

while True:
    changed = False
    
    if show_steps == True:
        print('') 

    # list the features to be evaluated
    excluded = list(set(X.columns) - set(included))
    
    if show_steps == True:
        print('(Step) Excluded = %s' % ', '.join(excluded))  

    # for each remaining feature to be evaluated
    for new_column in excluded:
        
        if show_steps == True:
            print('(Step) Trying %s...' % new_column)
            print('(Step) - Features = %s' % ', '.join(included + [new_column]))

        # fit the model with the Training data
        fit = model.fit(X_train[included + [new_column]], y_train) # fit a model; consider which predictors should be included
        # calculate the score (R^2 for Regression)
        r2 = model.score(X_test[included + [new_column]], y_test) # calculate the score
        # number of predictors in this model
        k = len(included + [new_column])
        # calculate the adjusted R^2
        adjusted_r2 = 1 - (((1 - r2)*(n-1)) / (n-k-1)) # calculate the Adjusted R^2

        if show_steps == True:
            print('(Step) - Adjusted R^2: This = %.3f; Best = %.3f' % 
                  (adjusted_r2, best['a_r2']))

        # if model improves
        if adjusted_r2 > best['a_r2']:
            # record new parameters
            best = {'feature': new_column, 'r2': r2, 'a_r2': adjusted_r2}
            # flag that found a better model
            changed = True
            if show_steps == True:
                print('(Step) - New Best!   : Feature = %s; R^2 = %.3f; Adjusted R^2 = %.3f' % 
                      (best['feature'], best['r2'], best['a_r2']))
    # END for

    # if found a better model after testing all remaining features
    if changed == True:
        # update control details
        included.append(best['feature'])
        excluded = list(set(excluded) - set(best['feature']))
        print('Added feature %-4s with R^2 = %.3f and adjusted R^2 = %.3f' % 
              (best['feature'], best['r2'], best['a_r2']))
    else:
        # terminate if no better model
        break

print('')
print('Resulting features:')
print(', '.join(included))


(Step) Excluded = pH, alcohol, total sulfur dioxide, density, free sulfur dioxide, volatile acidity, residual sugar, fixed acidity, red_wine, sulphates, chlorides, citric acid
(Step) Trying pH...
(Step) - Features = pH
(Step) - Adjusted R^2: This = -0.000; Best = 0.000
(Step) Trying alcohol...
(Step) - Features = alcohol
(Step) - Adjusted R^2: This = 0.215; Best = 0.000
(Step) - New Best!   : Feature = alcohol; R^2 = 0.216; Adjusted R^2 = 0.215
(Step) Trying total sulfur dioxide...
(Step) - Features = total sulfur dioxide
(Step) - Adjusted R^2: This = -0.002; Best = 0.215
(Step) Trying density...
(Step) - Features = density
(Step) - Adjusted R^2: This = 0.103; Best = 0.215
(Step) Trying free sulfur dioxide...
(Step) - Features = free sulfur dioxide
(Step) - Adjusted R^2: This = 0.005; Best = 0.215
(Step) Trying volatile acidity...
(Step) - Features = volatile acidity
(Step) - Adjusted R^2: This = 0.053; Best = 0.215
(Step) Trying residual sugar...
(Step) - Features = residual sugar
(S

(Step) - Adjusted R^2: This = 0.278; Best = 0.281
(Step) Trying red_wine...
(Step) - Features = alcohol, volatile acidity, density, free sulfur dioxide, total sulfur dioxide, sulphates, chlorides, red_wine
(Step) - Adjusted R^2: This = 0.280; Best = 0.281
(Step) Trying citric acid...
(Step) - Features = alcohol, volatile acidity, density, free sulfur dioxide, total sulfur dioxide, sulphates, chlorides, citric acid
(Step) - Adjusted R^2: This = 0.281; Best = 0.281
(Step) - New Best!   : Feature = citric acid; R^2 = 0.286; Adjusted R^2 = 0.281
Added feature citric acid with R^2 = 0.286 and adjusted R^2 = 0.281

(Step) Excluded = pH, fixed acidity, red_wine, residual sugar
(Step) Trying pH...
(Step) - Features = alcohol, volatile acidity, density, free sulfur dioxide, total sulfur dioxide, sulphates, chlorides, citric acid, pH
(Step) - Adjusted R^2: This = 0.281; Best = 0.281
(Step) Trying fixed acidity...
(Step) - Features = alcohol, volatile acidity, density, free sulfur dioxide, total 

In [11]:
# Fit model again with selected features

model.fit(X_train[included], y_train)

# Score model with test data

print("Best model has {} features ({}) with R2 score of {}".format(len(included), 
                                                                   ', '.join(included),
                                                                   round(model.score(X_test[included], y_test), 4)))

Best model has 8 features (alcohol, volatile acidity, density, free sulfur dioxide, total sulfur dioxide, sulphates, chlorides, citric acid) with R2 score of 0.2856
